# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [35]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

# Interact was not initially imported
from panel.interact import interact

In [12]:
# Read the Mapbox API key
load_dotenv("env_key.env")

map_box_api = os.getenv("Mapbox_key")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [13]:
# Import the necessary CSVs to Pandas DataFrames
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

In [14]:
neighborhood_avg_stats = (
    sfo_data.groupby(['neighborhood'],as_index=False).mean()
)

neighborhood_avg_stats.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
0,Alamo Square,366.020712,378401.0,2817.285714
1,Anza Vista,373.382198,379050.0,3031.833333
2,Bayview,204.588623,376454.0,2318.400000
3,Bayview Heights,590.792839,382295.0,3739.000000
4,Bernal Heights,576.746488,379374.5,3080.333333


In [15]:
top_neighborhood_sale_price = (
    neighborhood_avg_stats.nlargest(
        10,
        'sale_price_sqr_foot'
    ).reset_index().drop(columns='index')
)
top_neighborhood_sale_price.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
0,Union Square District,903.993258,377427.50,2555.166667
1,Merced Heights,788.844818,380348.00,3414.000000
2,Miraloma Park,779.810842,375967.25,2155.250000
3,Pacific Heights,689.555817,378401.00,2817.285714
4,Westwood Park,687.087575,382295.00,3959.000000


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [33]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    # Calculating the mean number of housing units per year
    yrly_units_avg = (
        sfo_data[['housing_units']].groupby('year').mean()
    )
    
    title = 'Housing Units in San Francisco from 2010 to 2016'
    yrly_housing_units = yrly_units_avg.hvplot.bar(
        x='year',
        y='housing_units',
        width=600,
        height=300,
        title=title,
        xlabel='Year',
        ylabel='Avg Housing Units',
        ylim = (370000,387500)
    ).opts(yformatter="%.0f")
    
    return yrly_housing_units


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # Calculate the average gross rent
    avg_gross_rent = (
        sfo_data[['gross_rent']].groupby('year').mean()
    )
    
    rent_title = 'Average Gross Rent in San Francisco'
    gross_rent_plot = avg_gross_rent.hvplot.line(
        x='year',
        y='gross_rent',
        width=800,
        height=400,
        title=rent_title,
        xlabel='Year',
        ylabel='Gross Rent'
    )
    
    return gross_rent_plot



def average_sales_price():
    """Average Sales Price Per Year."""
    
    # Calculate the average sale price per square foot
    avg_sale_price_sqft = (
        sfo_data[['sale_price_sqr_foot']].groupby('year').mean()
    )
    
    sqft_title = 'Average Sale Price per Square Foot in San Francisco'
    sale_price_plot = avg_sale_price_sqft.hvplot.line(
        x='year',
        y='sale_price_sqr_foot',
        width=800,
        height=400,
        title=sqft_title,
        xlabel='Year',
        ylabel='Avg. Sale Price'
    )
    
    return sale_price_plot



def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    sales_price_sqr_foot = (sfo_data.reset_index().groupby(['year','neighborhood'],as_index=False).mean())
    
    def average_price_neighborhood(neighborhood):
    
        neighborhood_prices = sales_price_sqr_foot.loc[sales_price_sqr_foot['neighborhood']==neighborhood]

        price_sqr_foot_title = f'{neighborhood} Average Sale Price'

        neighborhood_avg_price = neighborhood_prices.hvplot.line(
                x='year',
                y='sale_price_sqr_foot',
                width=800,
                height=400,
                title=price_sqr_foot_title,
                xlabel='Year',
                ylabel='Avg. Sale Price per Square Foot'
            )

        return neighborhood_avg_price
    
    # Use interact function to create interaction widget
    list_of_neighborhoods = sales_price_sqr_foot['neighborhood']

    interact(average_price_neighborhood, neighborhood=list_of_neighborhoods)
    
    return interact(average_price_neighborhood, neighborhood=list_of_neighborhoods)




def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    # Calculate top ten neighborhoods by average sale price
    top_neighborhood_sale_price = (
        neighborhood_avg_stats.nlargest(
            10,
            'sale_price_sqr_foot'
        ).reset_index().drop(columns='index')
    )
    
    top_ten_title = 'Top 10 Expensive Neighborhoods in SFO'
    top_ten_plot = top_neighborhood_sale_price.hvplot.bar(
        rot=90,x='neighborhood',
        y='sale_price_sqr_foot',
        width=800,
        height=400,
        title=top_ten_title
    )
    
    return top_ten_plot



def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    # Calculate top ten neighborhoods by average sale price
    rent_sale_comparison = (
        neighborhood_avg_stats.nlargest(
            10,
            'sale_price_sqr_foot'
        ).reset_index().drop(columns='index')
    )
    
    rent_sale_title ="Rent vs Sales comparison of SFO neighborhood per year"
    rent_sale_plot = top_neighborhood_sale_price.hvplot.bar(
        stacked=False, groupby='neighborhood',height=500, rot=60,
        title="Rent vs Sales comparison of SFO neighborhood per year")
    
    return top_ten_plot

    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    # YOUR CODE HERE!
    
    parallel_coordinates_plot = px.parallel_coordinates(
    top_neighborhood_sale_price,
    dimensions=['sale_price_sqr_foot','housing_units','gross_rent'],
    color='sale_price_sqr_foot',
    color_continuous_scale=px.colors.sequential.Inferno,
    labels={'sale_price_sqr_foot': 'Sale Price per Sq.ft.',
            'housing_units': 'Housing Units',
            'gross_rent': 'Gross Rent',
    },
    width=800
    )
    
    return parallel_coordinates_plot


def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!
    
    parallel_categories_plot = px.parallel_categories(
    top_neighborhood_sale_price,
    dimensions=['neighborhood','sale_price_sqr_foot','housing_units','gross_rent'],
    color='sale_price_sqr_foot',
    color_continuous_scale=px.colors.sequential.Inferno,
    labels={
            'neighborhood':'Neighborhood',
            'sale_price_sqr_foot': 'Sale Price per Sq.ft.',
            'housing_units': 'Housing Units',
            'gross_rent': 'Gross Rent',
    },
    width=800
    )
    
    return parallel_categories_plot



def neighborhood_map():
    """Neighborhood Map."""

    neighborhood_stats_avg = (
        sfo_data.groupby('neighborhood',as_index=False).mean()
    )
    
    neighborhood_stats_coord = pd.concat(
        [df_neighborhood_locations, neighborhood_stats_avg],
        axis="columns",
        join="inner"
    ).drop(columns='neighborhood')

def sunburst():
    """Sunburst Plot."""
    
    map_title='Cost Analysis of the most expensive nieghborhoods in San Francisco per year'
    sunplot_plot = px.sunburst(
        df,
        path=['year','sale_price_sqr_foot'],
        values='gross_rent',
        color='year',
        title=title
        
    )
    
    return mapbox_plot


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [36]:
# Create panels to structure the layout of the dashboard
housing_units_column = pn.Column(
    '## Housing Units Sold per Year',
    housing_units_per_year()
)

averages_column = pn.Column(
    'Avg. Gross Rent, Sales Price, and Price by Neighborhood',
    average_gross_rent(),
    average_sales_price(),
    average_price_by_neighborhood()
)

top_ten_column = pn.Column(
    'Top Ten Most Expensive Neighborhoods',
    top_most_expensive_neighborhoods(),
)

parallel_column = pn.Column(
    '## Parallel Coordinates and Categories',
    parallel_coordinates(),
    parallel_categories(),
)

map_column = pn.Column(
    '## Average Values per Neighborhood Maps',
    neighborhood_map()
)

# Create tabs
sfo_dashboard = pn.Tabs(
    ('Housing Units Sold', housing_units_column),
    ('Averages', averages_column),
    ('Most Expensive', top_ten_column),
    ('Parallel Graphs', parallel_column),
    ('Map', map_column)
)

## Serve the Panel Dashboard

In [37]:
# Serve the# dashboard
sfo_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
        [3] Column
            [0] Column
                [0] Select(name='neighborhood', options=['Alamo Square', ...], value='Alamo Square')
            [1] Row
                [0] HoloViews(Curve, name='interactive02453')
    [2] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [4] Column
        [0] Markdown(str)
        [1] Str(None)